In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
%matplotlib inline
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# The following is to downlaod and use pretrained models from the model-zoo
import torchvision.models as models
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
# Hyperparameters
batch_size = 256
# Learning rate
lr = 0.001
# Number of training epochs
num_epochs = 2
# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Path to dataset
dataset_root_path = '/scratch2/datasets/cifar10/'
# dataset_root_path = '/content/gdrive/My Drive/cifar10/'

The output of torchvision datasets are PILImage images of range [0, 1].
We transform them to Tensors of normalized range [-1, 1].



In [4]:
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
trans = transforms.Compose([transforms.Resize(224),
                               transforms.CenterCrop(224),
                               transforms.ToTensor(),
                               normalize])

trainset = torchvision.datasets.CIFAR10(root=dataset_root_path, train=True,
                                        download=True, transform=trans)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root=dataset_root_path, train=False,
                                       download=True, transform=trans)
testloader = torch.utils.data.DataLoader(testset, batch_size=2*batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
print(trainset.train_data.shape)
print(testset.test_data.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [6]:
# Download the pretrained Resnet50 model and load it to the device (GPU)
resnet50 = models.resnet50(pretrained=True)
resnet50 = resnet50.to(device)

In [7]:
# Freeze the layers
for param in resnet50.parameters():
    param.requires_grad = False

In [8]:
# Change the last layer to cifar10 number of output classes.
# Also unfreeze the penultimate layer. We will finetune just these two layers.
resnet50.fc = nn.Sequential(
                      nn.Linear(2048, 256), 
                      nn.ReLU(), 
                      nn.Linear(256, 10)
)


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet50.parameters(), lr=0.001)

4. Train the network

In [10]:
resnet50 = resnet50.to(device)
for epoch in range(num_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    pbar = tqdm(trainloader)
    i = 0
    for data in pbar:
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description("Processing epoch {:d} minibatch {:d} train loss {:.3f}".format(epoch,\
                                                            i+1, running_loss/(i+1)))
        i += 1

print('Finished Training')

Processing epoch 0 minibatch 196 train loss 0.778: 100%|██████████| 196/196 [01:34<00:00,  2.56it/s]
Processing epoch 1 minibatch 196 train loss 0.568: 100%|██████████| 196/196 [01:34<00:00,  2.56it/s]

Finished Training


5. Test the network on the test data

In [14]:
correct = 0
total = 0
i = 0
with torch.no_grad():
    pbar = tqdm(testloader)
    for data in pbar:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = resnet50(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        pbar.set_description("minibatch {:d} test accuracy {:4.2f}%".format(i+1,\
                                                            100.0*correct/total))
        i += 1

print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100.0 * correct / total))

minibatch 20 test accuracy 79.12%: 100%|██████████| 20/20 [00:19<00:00,  1.22it/s]

Accuracy of the network on the 10000 test images: 79.12 %
